<a href="https://colab.research.google.com/github/tezike/vaccination_zindi/blob/master/00_vaccination_tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TODO:
Mixed precision

kfold

use another tokenizer

use higher seq_len

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
%%capture
!pip install git+https://github.com/fastai/fastai2.git -q
# !pip install git+https://github.com/fastai/fastcore.git -q

!pip install -q sentencepiece

# # Upload utils.py from fastai repository
# !wget https://raw.githubusercontent.com/fastai/fastbook/master/utils.py

MessageError: ignored

## Colab_setup

In [0]:
from pathlib import Path
import os
from google.colab import drive

In [0]:
def create_path(path):
    if not os.path.isdir(path):
        path.mkdir(parents=True, exist_ok=True)
    return path

In [0]:
from google.colab import drive
drive.mount('/content/drive')
root_dir = Path('/content/drive/My Drive')
base_path = create_path(root_dir/'Bert_zindi')
base_path

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


PosixPath('/content/drive/My Drive/Bert_zindi')

In [0]:
colab_path = Path('/content')

In [0]:
data_path = create_path(base_path/'dataset')

In [0]:
model_path = create_path(base_path/'models')

In [0]:
bert_path = (create_path(colab_path/'input/bert_uncased'))

In [0]:
from fastai2.text.all import *

In [0]:
bs = 64
seq_len = 72

In [0]:
train_df = pd.read_csv(data_path/'train.csv')
test_df = pd.read_csv(data_path/'test.csv')
len(train_df), len(test_df)

(10001, 5177)

In [0]:
train_df.dropna(axis='rows', inplace=True)

In [0]:
train_df['label'].value_counts()

 0.0    4908
 1.0    4053
-1.0    1038
Name: label, dtype: int64

## DataLoaders

In [0]:
tok = SentencePieceTokenizer

In [0]:
tweet_lm = DataBlock(
            blocks=TextBlock.from_df(text_cols='safe_text', is_lm=True, tok_func=tok),
            get_x=ColReader('text'),
            splitter=RandomSplitter(valid_pct=0.15, seed=42)
            )

In [0]:
df = pd.concat([train_df, test_df])

In [0]:
dls = tweet_lm.dataloaders(source=df, bs=int(bs), seq_len=int(seq_len))

In [0]:
dls.show_batch(max_n=3)

In [0]:
len(dls.train_ds.vocab)

## Language Learner

In [0]:
learn_lm = language_model_learner(dls, arch=AWD_LSTM, drop_mult=0.3, metrics=[accuracy, Perplexity()])

In [0]:
learn_lm.lr_find()

In [0]:
lm_lr = 4e-2

In [0]:
learn_lm.fit_one_cycle(3, lm_lr)

In [0]:
learn.unfreeze()

In [0]:
learn.lr_find()

In [0]:
learn_lm.fit_one_cycle(3, lm_lr)

In [0]:
learn_lm.export(model_path/'lm_trained.pkl')

In [0]:
learn_lm = load_learner(model_path, 'lm_trained.pkl')

In [0]:
learn_lm.dls = dls

In [0]:
learn.path = model_path

In [0]:
learn_lm.save_encoder('lm_encoder')

## Classification

In [0]:
train_df.agreement = train_df.agreement.apply(lambda x: round(x, 2))
train_df = train_df.query('agreement != 0.33')
train_df = train_df[['safe_text', 'label']]

In [0]:
train_df.rename(columns={'safe_text':'text'}, inplace=True)
test_df.rename(columns={'safe_text':'text'}, inplace=True)

In [0]:
test_df.fillna(' ', inplace=True)

kfold

In [0]:
from sklearn.model_selection import StratifiedKFold

In [0]:
# sample the full data to add shuffling
train_df = train_df.sample(frac=1.)

n_splits=5

# use 5 folds
kf = StratifiedKFold(n_splits=n_splits, random_state=42)

preds = []

for fold, (train_idx, valid_idx) in enumerate(kf.split(X=train_df, y=train_df.label.values)):
    print(len(train_idx), len(valid_idx), fold)
    print(train_idx)
    break
    # splits = IndexSplitter(val_idx)
    # split = splits(train_df)
    # split_list = [split[0], split[1]]

    tweet_cls = DataBlock(blocks=[TextBlock.from_df(text_cols='text', vocab=dls.vocab, tok_func=tok),
                              RegressionBlock],
                      get_x=ColReader(cols='text'),
                      get_y=ColReader(cols='label'),
                      splitter=IndexSplitter(val_idx)


    tweet_cls_dls = tweet_cls.dataloaders(source=train_df, verbose=True,
                                      bs=int(bs),
                                      seq_len=int(seq_len))
    
    learn_cls = text_classifier_learner(dls=tweet_cls_dls,
                                    arch=AWD_LSTM, loss_func=mse,
                                    metrics=[accuracy, error_rate, rmse], y_range=[-1, 1])
    
    learn_cls.path = model_path

    learn_cls.load_encoder('lm_encoder')

    # learn.lr_find()

    learn_lm.fit_one_cycle(3, )

    learn_lm.unfreeze()

    # learn.lr_find()

    learn_lm.fit_one_cycle(3, )

    test_dls = learn_cls.dls.test_dl(test_items=test_df[['text']])

    preds, _, preds_raw = learn_cls.get_preds(dl=_test_dl, with_decoded=True)

    learn.append(preds.flatten())    

7999 2000 0
[1964 1969 1970 ... 9996 9997 9998]


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [0]:
tweet_cls = DataBlock(blocks=[TextBlock.from_df(text_cols='text', vocab=dls.vocab, tok_func=tok),
                              RegressionBlock],
                      get_x=ColReader(cols='text'),
                      get_y=ColReader(cols='label'),
                      splitter=RandomSplitter(valid_pct=0.2, seed=42))

In [0]:
tweet_cls_dls = tweet_cls.dataloaders(source=train_df, verbose=True,
                                      bs=int(bs),
                                      seq_len=int(seq_len))

Setting up after_item: Pipeline: ToTensor
Setting up before_batch: Pipeline: partial
Setting up after_batch: Pipeline: 


In [0]:
tweet_cls_dls.show_batch(max_n=3)

,text,text_
0,"xxbos < user > xxmaj how xxmaj could xxmaj you xxmaj put xxmaj ur xxmaj child xxmaj in xxmaj so xxmaj much xxmaj danger ? xxmaj most xxmaj of xxmaj the xxmaj diseases xxmaj that xxmaj they xxmaj have xxmaj vaccines 4 , i xxmaj had xxmaj xxunk . xxmaj vaccinate . xxunk",1.0
1,"xxbos ( ( cada xxmaj dia ) ) xxmaj measles is xxmaj not a xxmaj mild xxmaj disease : xxmaj measles xxmaj is xxmaj not a xxmaj mild xxmaj disease i had measles when i was eight years old . xxmaj in those days , xxmaj ma …",0.0
2,"xxbos < user > < user > xxmaj ppl xxmaj need 2 xxmaj immunize xxmaj their xxmaj kids . xxmaj do xxmaj research . xxmaj most xxmaj diseases xxup worse xxup than xxup inoculation . xxup stop xxmaj this xxmaj misguided xxmaj insanity , xxmaj shots !",1.0


In [0]:
learn_cls = text_classifier_learner(dls=tweet_cls_dls,
                                    arch=AWD_LSTM, loss_func=mse,
                                    metrics=[accuracy, error_rate, rmse], y_range=[-1, 1])

In [0]:
learn_cls.load_encoder(f'2_4e-2_lm_fine_tuned_enc')

In [0]:
# learn_cls.model

In [0]:
learn_cls.lr_find()

In [0]:
learn_cls.fine_tune(epochs=4, base_lr=3e-3)

In [0]:
learn_cls.recorder.plot_loss()

## Inference

In [0]:
_test_dl = learn_cls.dls.test_dl(test_items=test_df[['text']])

In [0]:
preds, _, preds_raw = learn_cls.get_preds(dl=_test_dl, with_decoded=True)

In [0]:
preds

tensor([[-0.2676],
        [ 0.6066],
        [ 0.4288],
        ...,
        [ 0.4624],
        [ 0.7899],
        [ 0.2505]])

In [0]:
sample = pd.read_csv(vaccination/'sub.csv')
sample.loc[:, 'label'] = preds
sample.to_csv("submission.csv", index=False)